## Ideia basica para o projeto... apenas um rascunho com a interface

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
import xml.etree.ElementTree as ET

class Visualizador:
    def __init__(self, root):
        self.root = root
        self.root.title("Visualizador de Objetos 2D")
        
        # Configurar o menu
        menu = tk.Menu(root)
        root.config(menu=menu)
        file_menu = tk.Menu(menu)
        menu.add_cascade(label="Arquivo", menu=file_menu)
        file_menu.add_command(label="Abrir", command=self.abrir_arquivo)
        file_menu.add_command(label="Gerar XML de Saída", command=self.gerar_xml_saida)
        
        # Frame principal para conter canvas e minimapa
        frame_principal = tk.Frame(root)
        frame_principal.pack(fill="both", expand=True)
        
        # Desenhar as viewports
        # estes dados deverão ser lidos do arquivo de entrada...
        largura = 800
        altura = 600
        # canvas da Viewport principal
        self.canvas = tk.Canvas(frame_principal, width=largura, height=altura, bg="white")
        self.canvas.pack(side="left", fill="both", expand=True)
        
        # Canvas do minimapa na lateral direita
        # usar um tamanho fixo para o minimapa
        self.minimap = tk.Canvas(frame_principal, width=150, height=120, bg="lightgrey") # razão de aspecto 4:3
        self.minimap.pack(side="right", padx=10, pady=10)

        # lista de objetos lidos do XML
        self.objects = []           
        self.window = {'wmin':(0.0,0.0), 'wmax':(1.0,1.0)}
        self.viewport = {'vpmin':(0.0,0.0), 'vpmax':(largura, altura)}
        # matriz 3x3 world->viewport
        self.M = [[1.0,0.0,0.0],[0.0,1.0,0.0],[0.0,0.0,1.0]]
        # limites do mundo baseados nos objetos
        self.world_bounds = {'xmin': 0.0, 'xmax': 1.0, 'ymin': 0.0, 'ymax': 1.0}
        # caminho do arquivo carregado
        self.caminho_arquivo = None

        # função seta para mover a window
        root.bind("<Left>", lambda e: self._mover_e_recarregar(-1,0))
        root.bind("<Right>", lambda e: self._mover_e_recarregar(1,0))
        root.bind("<Up>", lambda e: self._mover_e_recarregar(0,1))
        root.bind("<Down>", lambda e: self._mover_e_recarregar(0,-1))

    # apenas abrir o arquivo e chamar a função para carregar seus dados...
    def abrir_arquivo(self):
        caminho = filedialog.askopenfilename(filetypes=[("XML files","*.xml"),("All files","*.*")])
        if not caminho:
            return
        try:
            self.caminho_arquivo = caminho  # armazena o caminho do arquivo
            self.carregar_arquivo(caminho)
        except Exception as e:
            print("Erro ao carregar arquivo:", e)

    # calcula os limites do mundo baseado nos objetos
    def calcular_limites_mundo(self):
        if not self.objects:
            return
        
        xs = []
        ys = []
        for obj in self.objects:
            for x, y in obj['points']:
                xs.append(x)
                ys.append(y)
        
        if xs and ys:
            # define os limites exatos dos objetos
            xmin, xmax = min(xs), max(xs)
            ymin, ymax = min(ys), max(ys)
            
            # calcula o tamanho atual da window para manter proporção
            wwidth = self.window['wmax'][0] - self.window['wmin'][0]
            wheight = self.window['wmax'][1] - self.window['wmin'][1]
            
            # adiciona margem baseada no tamanho da window atual
            margin_x = wwidth * 0.3  # margem de 30% do tamanho da window
            margin_y = wheight * 0.3
            
            self.world_bounds = {
                'xmin': xmin - margin_x,
                'xmax': xmax + margin_x,
                'ymin': ymin - margin_y,
                'ymax': ymax + margin_y
            }

    # ler os dados do arquivo aberto
    def carregar_arquivo(self, caminho):
        tree = ET.parse(caminho)
        root = tree.getroot()

        # parse viewport
        vp = root.find('viewport')
        if vp is not None:
            vpmin = vp.find('vpmin')
            vpmax = vp.find('vpmax')
            if vpmin is not None and vpmax is not None:
                self.viewport['vpmin'] = (float(vpmin.get('x', '0')), float(vpmin.get('y', '0')))
                self.viewport['vpmax'] = (float(vpmax.get('x', '800')), float(vpmax.get('y', '600')))

        # ajusta canvas ao tamanho do viewport lido
        vpxmin, vpymin = self.viewport['vpmin']
        vpxmax, vpymax = self.viewport['vpmax']
        largura = max(200, int(abs(vpxmax - vpxmin)))
        altura = max(200, int(abs(vpymax - vpymin)))
        self.canvas.config(width=largura, height=altura)

        # parse window
        w = root.find('window')
        if w is not None:
            wmin = w.find('wmin')
            wmax = w.find('wmax')
            if wmin is not None and wmax is not None:
                self.window['wmin'] = (float(wmin.get('x','0')), float(wmin.get('y','0')))
                self.window['wmax'] = (float(wmax.get('x','1')), float(wmax.get('y','1')))

        # parse geometric objects
        self.objects = []
        for child in root:
            tag = child.tag.lower()
            if tag == 'ponto':
                x = float(child.get('x','0'))
                y = float(child.get('y','0'))
                self.objects.append({'type':'ponto', 'points':[(x,y)]})
            elif tag == 'reta':
                pts = []
                for p in child.findall('ponto'):
                    pts.append((float(p.get('x','0')), float(p.get('y','0'))))
                if len(pts) >= 2:
                    self.objects.append({'type':'reta', 'points':pts[:2]})
            elif tag == 'poligono':
                pts = []
                for p in child.findall('ponto'):
                    pts.append((float(p.get('x','0')), float(p.get('y','0'))))
                if len(pts) >= 3:
                    self.objects.append({'type':'poligono', 'points':pts})

        # calcula os limites do mundo baseado nos objetos carregados
        self.calcular_limites_mundo()

        # monta matriz de transformação world -> viewport
        self.recalcular_matriz()

        # desenha
        self.desenhar_viewport()
        self.desenhar_minimapa()
    
    # recalcula a matriz de transformação mantendo proporções
    def recalcular_matriz(self):
        wxmin, wymin = self.window['wmin']
        wxmax, wymax = self.window['wmax']
        vpxmin, vpymin = self.viewport['vpmin']
        vpxmax, vpymax = self.viewport['vpmax']

        # calcula fatores de escala
        sx = (vpxmax - vpxmin) / (wxmax - wxmin) if (wxmax - wxmin) != 0 else 1.0
        sy = (vpymax - vpymin) / (wymax - wymin) if (wymax - wymin) != 0 else 1.0

        # aplica escala e traduz ou seja ela inverte y para coordenadas do widget canvas do tkinter
        self.M = [
            [sx, 0.0, vpxmin - sx * wxmin],
            [0.0, -sy, vpymin + sy * wymax],
            [0.0, 0.0, 1.0]
        ]
    
    # aplica a transformada de viewport a um ponto e retornar o ponto calculado
    def window2viewport(self, ponto, window, viewport):
        xw, yw = ponto
        # assume M atual corresponde a window/viewport atuais
        xp = self.M[0][0] * xw + self.M[0][1] * yw + self.M[0][2]
        yp = self.M[1][0] * xw + self.M[1][1] * yw + self.M[1][2]
        # converte para coordenadas internas do canvas subtraindo vpmin
        vpxmin, vpymin = viewport['vpmin']
        return (xp - vpxmin, yp - vpymin)

    # redensenhar a viewport
    def desenhar_viewport(self):
        self.canvas.delete("all")
        largura = int(self.canvas.cget('width'))
        altura = int(self.canvas.cget('height'))

        # desenha objetos
        for obj in self.objects:
            if obj['type'] == 'ponto':
                xw, yw = obj['points'][0]
                cx, cy = self.window2viewport((xw,yw), self.window, self.viewport)
                r = 3
                self.canvas.create_oval(cx-r, cy-r, cx+r, cy+r, fill='black')
            elif obj['type'] == 'reta':
                (x0,y0),(x1,y1) = obj['points'][0], obj['points'][1]
                c0x, c0y = self.window2viewport((x0,y0), self.window, self.viewport)
                c1x, c1y = self.window2viewport((x1,y1), self.window, self.viewport)
                self.canvas.create_line(c0x, c0y, c1x, c1y, width=2)
            elif obj['type'] == 'poligono':
                pts = []
                for (x,y) in obj['points']:
                    px, py = self.window2viewport((x,y), self.window, self.viewport)
                    pts.extend([px, py])
                if len(pts) >= 6:
                    # fecha o polígono
                    self.canvas.create_line(*pts, pts[0], pts[1], width=2)

        # borda da viewport
        self.canvas.create_rectangle(0,0,largura-1, altura-1, outline='black')

        # mostra a matriz no canto superior esquerdo
        txt = (f"Matriz 3x3 (world->viewport):\n"
               f"[{self.M[0][0]:.4f} {self.M[0][1]:.4f} {self.M[0][2]:.4f}]\n"
               f"[{self.M[1][0]:.4f} {self.M[1][1]:.4f} {self.M[1][2]:.4f}]\n"
               f"[{self.M[2][0]:.4f} {self.M[2][1]:.4f} {self.M[2][2]:.4f}]")
        self.canvas.create_text(8, 8, anchor='nw', text=txt, font=("Courier", 9), fill='blue')

    # desenhar a viewport do minimapa
    def desenhar_minimapa(self):
        self.minimap.delete("all")
        # calcula o bounding box do mundo com base nos objetos e na window
        xs = []
        ys = []
        for obj in self.objects:
            for x,y in obj['points']:
                xs.append(x); ys.append(y)
        xs.extend([self.window['wmin'][0], self.window['wmax'][0]])
        ys.extend([self.window['wmin'][1], self.window['wmax'][1]])
        xs.extend([self.world_bounds['xmin'], self.world_bounds['xmax']])
        ys.extend([self.world_bounds['ymin'], self.world_bounds['ymax']])

        if not xs:
            return

        xmin, xmax = min(xs), max(xs)
        ymin, ymax = min(ys), max(ys)
        if xmin == xmax:
            xmin -= 1; xmax += 1
        if ymin == ymax:
            ymin -= 1; ymax += 1

        # padding
        pad_x = (xmax - xmin) * 0.08
        pad_y = (ymax - ymin) * 0.08
        xmin -= pad_x; xmax += pad_x; ymin -= pad_y; ymax += pad_y

        mw = int(self.minimap.cget('width'))
        mh = int(self.minimap.cget('height'))

        def world_to_minimap(px, py):
            sx = mw / (xmax - xmin)
            sy = mh / (ymax - ymin)
            mx = (px - xmin) * sx
            my = mh - (py - ymin) * sy  # inverte y
            return mx, my

        # desenha limites do mundo como retângulo cinza
        p1 = world_to_minimap(self.world_bounds['xmin'], self.world_bounds['ymin'])
        p2 = world_to_minimap(self.world_bounds['xmin'], self.world_bounds['ymax'])
        p3 = world_to_minimap(self.world_bounds['xmax'], self.world_bounds['ymax'])
        p4 = world_to_minimap(self.world_bounds['xmax'], self.world_bounds['ymin'])
        self.minimap.create_polygon(p1[0],p1[1], p2[0],p2[1], p3[0],p3[1], p4[0],p4[1],
                                    outline='gray', fill='lightgray', width=1)

        # desenha objetos
        for obj in self.objects:
            if obj['type'] == 'ponto':
                x,y = obj['points'][0]
                mx,my = world_to_minimap(x,y)
                self.minimap.create_oval(mx-2, my-2, mx+2, my+2, fill='black')
            elif obj['type'] == 'reta':
                (x0,y0),(x1,y1) = obj['points'][0], obj['points'][1]
                m0 = world_to_minimap(x0,y0); m1 = world_to_minimap(x1,y1)
                self.minimap.create_line(m0[0], m0[1], m1[0], m1[1], width=1)
            elif obj['type'] == 'poligono':
                pts = []
                for x,y in obj['points']:
                    mx,my = world_to_minimap(x,y)
                    pts.extend([mx,my])
                if len(pts) >= 6:
                    self.minimap.create_line(*pts, pts[0], pts[1], width=1)

        # desenha o quadrado de foco da window atual no minimapa
        wxmin, wymin = self.window['wmin']
        wxmax, wymax = self.window['wmax']
        p1 = world_to_minimap(wxmin, wymin)
        p2 = world_to_minimap(wxmin, wymax)
        p3 = world_to_minimap(wxmax, wymax)
        p4 = world_to_minimap(wxmax, wymin)
        self.minimap.create_polygon(p1[0],p1[1], p2[0],p2[1], p3[0],p3[1], p4[0],p4[1],
                                    outline='red', fill='', width=2)

    # move window e redesenha
    def _mover_e_recarregar(self, dx, dy):
        wminx, wminy = self.window['wmin']
        wmaxx, wmaxy = self.window['wmax']
        
        # calcula o tamanho atual da window
        window_width = wmaxx - wminx
        window_height = wmaxy - wminy
        
        # calcula as novas coordenadas propostas
        new_wminx = wminx + dx
        new_wminy = wminy + dy
        new_wmaxx = new_wminx + window_width
        new_wmaxy = new_wminy + window_height
        
        # verifica se os limites do mundo são maiores que a window
        world_width = self.world_bounds['xmax'] - self.world_bounds['xmin']
        world_height = self.world_bounds['ymax'] - self.world_bounds['ymin']
        
        # só aplica limitações se o mundo for maior que a window
        if world_width > window_width:
            # limita movimento horizontal
            if new_wminx < self.world_bounds['xmin']:
                new_wminx = self.world_bounds['xmin']
                new_wmaxx = new_wminx + window_width
            elif new_wmaxx > self.world_bounds['xmax']:
                new_wmaxx = self.world_bounds['xmax']
                new_wminx = new_wmaxx - window_width
        else:
            # centraliza a window no mundo se for menor
            center_x = (self.world_bounds['xmin'] + self.world_bounds['xmax']) / 2
            new_wminx = center_x - window_width / 2
            new_wmaxx = center_x + window_width / 2
        
        if world_height > window_height:
            # limita movimento vertical
            if new_wminy < self.world_bounds['ymin']:
                new_wminy = self.world_bounds['ymin']
                new_wmaxy = new_wminy + window_height
            elif new_wmaxy > self.world_bounds['ymax']:
                new_wmaxy = self.world_bounds['ymax']
                new_wminy = new_wmaxy - window_height
        else:
            # centraliza a window no mundo se for menor
            center_y = (self.world_bounds['ymin'] + self.world_bounds['ymax']) / 2
            new_wminy = center_y - window_height / 2
            new_wmaxy = center_y + window_height / 2
        
        # atualiza a window com os valores limitados
        self.window['wmin'] = (new_wminx, new_wminy)
        self.window['wmax'] = (new_wmaxx, new_wmaxy)

        # recalcula matriz com nova window
        self.recalcular_matriz()
        self.desenhar_viewport()
        self.desenhar_minimapa()

    # transformar pontos do mundo para viewport usando a matriz atual
    def transformar_ponto_para_viewport(self, x, y):
        """Aplica a transformação de viewport a um ponto usando a matriz atual"""
        xp = self.M[0][0] * x + self.M[0][1] * y + self.M[0][2]
        yp = self.M[1][0] * x + self.M[1][1] * y + self.M[1][2]
        return (xp, yp)

    # transformar todos os objetos para coordenadas da viewport
    def transformar_objetos_para_viewport(self):
        """Transforma todos os objetos para coordenadas da viewport"""
        objetos_transformados = []
        
        for obj in self.objects:
            obj_transformado = {'type': obj['type'], 'points': []}
            
            for x, y in obj['points']:
                xp, yp = self.transformar_ponto_para_viewport(x, y)
                obj_transformado['points'].append((xp, yp))
            
            objetos_transformados.append(obj_transformado)
        
        return objetos_transformados

    # função para formatar XML com indentação
    def _indent_xml(self, elem, level=0):
        """Adiciona indentação ao XML para formatação adequada"""
        i = "\n" + level * "\t"
        if len(elem):
            if not elem.text or not elem.text.strip():
                elem.text = i + "\t"
            if not elem.tail or not elem.tail.strip():
                elem.tail = i
            for child in elem:
                self._indent_xml(child, level + 1)
            if not child.tail or not child.tail.strip():
                child.tail = i
        else:
            if level and (not elem.tail or not elem.tail.strip()):
                elem.tail = i

    # gerar XML de saída com dados originais e transformados
    def gerar_xml_saida(self):
        """Gera arquivo XML de saída com dados originais e transformados"""
        if not hasattr(self, 'caminho_arquivo') or not self.caminho_arquivo:
            messagebox.showwarning("Aviso", "Nenhum arquivo foi carregado ainda!")
            return
        
        try:
            # solicita onde salvar o arquivo
            caminho_saida = filedialog.asksaveasfilename(
                defaultextension=".xml",
                filetypes=[("XML files", "*.xml"), ("All files", "*.*")],
                title="Salvar XML de Saída"
            )
            
            if not caminho_saida:
                return
            
            # carrega o arquivo original
            tree = ET.parse(self.caminho_arquivo)
            root = tree.getroot()
            
            # obtém objetos transformados
            objetos_transformados = self.transformar_objetos_para_viewport()
            
            # adiciona comentário separador
            comentario = ET.Comment(" Dados transformados para coordenadas da viewport ")
            root.append(comentario)
            
            # adiciona objetos transformados ao XML
            for obj_trans in objetos_transformados:
                if obj_trans['type'] == 'ponto':
                    x, y = obj_trans['points'][0]
                    ponto_elem = ET.SubElement(root, 'ponto')
                    ponto_elem.set('x', f"{x:.6f}")
                    ponto_elem.set('y', f"{y:.6f}")
                    
                elif obj_trans['type'] == 'reta':
                    reta_elem = ET.SubElement(root, 'reta')
                    for x, y in obj_trans['points']:
                        ponto_elem = ET.SubElement(reta_elem, 'ponto')
                        ponto_elem.set('x', f"{x:.6f}")
                        ponto_elem.set('y', f"{y:.6f}")
                        
                elif obj_trans['type'] == 'poligono':
                    poligono_elem = ET.SubElement(root, 'poligono')
                    for x, y in obj_trans['points']:
                        ponto_elem = ET.SubElement(poligono_elem, 'ponto')
                        ponto_elem.set('x', f"{x:.6f}")
                        ponto_elem.set('y', f"{y:.6f}")
            
            # aplica formatação com indentação
            self._indent_xml(root)
            
            # salva o arquivo
            tree.write(caminho_saida, encoding='utf-8', xml_declaration=True)
            messagebox.showinfo("Sucesso", f"XML de saída gerado com sucesso!\nSalvo em: {caminho_saida}")
            
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao gerar XML de saída: {e}")
        
  
if __name__ == "__main__":
    root = tk.Tk()
    app = Visualizador(root)
    root.mainloop()